## Pokeapi - CoderHouse
Jupyter Notebook com as execuções de entrega do projeto, respeitando:
1. Coleta de dados de uma API
2. Tratamento desses dados
3. Armazenamento em banco de dados

#### Começamos importando as bibliotecas e funções
1. pandas - Para analise dos dados coletados
2. src.utils - Arquivo com as funções criadas para conexão na API, coleta, tratamento e armazenamento dos dados
3. src.connect_database - Usado para fazer a conexão com o banco de dados.
4. src.config - Onde esta as queries de "input" de dados no banco, assim como as informações de conesão.  

In [1]:
import pandas as pd
from src.utils import *
from src.connect_database import *
from src.config import QUERY_INSERT

### Inicia fazendo a conexão na API
A página inicial traz uma série de URLs para cada Pokemon

In [2]:
pokemon = get_all_pokemons_from_api()
pokemon

'https://pokeapi.co/api/v2/pokemon/1/'

### Com a URL do Pokemon, inicia o tratamento dos dados

Coleta as informações do Pokemon

In [3]:
pokemon_info = get_pokemon_info(pokemon)

client_db = connect_to_mysql()
query_pokemon = QUERY_INSERT['pokemon']
store_pokemon_data(pokemon_info, client_db, query_pokemon)
disconnect_to_mysql(client_db)

df_pokemon_info = pd.DataFrame(pokemon_info)
df_pokemon_info

[connect_to_mysql] Connection success to MySQL!
[disconnect_to_mysql] Client disconnected!


,pokemon_id,name,height,weight
0,1,bulbasaur,7,69


Coleta as informações de habilidades

In [4]:
pokemon_abilities = get_abilities_from_api(pokemon)

client_db = connect_to_mysql()
query_abilities = QUERY_INSERT['abilities']
store_pokemon_data(pokemon_abilities, client_db, query_abilities)
disconnect_to_mysql(client_db)

df_pokemon_abilities = pd.DataFrame(pokemon_abilities)
df_pokemon_abilities

[connect_to_mysql] Connection success to MySQL!
[disconnect_to_mysql] Client disconnected!


,abilitie_id,name,effect
0,65,overgrow,When this Pokémon has 1/3 or less of its HP re...
1,34,chlorophyll,Während strong sunlight ist die speed eines Po...


Coleta as informações dos movimentos

In [5]:
pokemon_moves = get_moves_from_api(pokemon)

client_db = connect_to_mysql()
query_moves = QUERY_INSERT['moves']
store_pokemon_data(pokemon_moves, client_db, query_moves)
disconnect_to_mysql(client_db)

df_pokemon_moves = pd.DataFrame(pokemon_moves)
df_pokemon_moves

[connect_to_mysql] Connection success to MySQL!
[disconnect_to_mysql] Client disconnected!


,move_id,name,effect
0,13,razor-wind,Inflicts regular damage. User's critical hit ...
1,14,swords-dance,Raises the user's Attack by two stages.
2,15,cut,Inflicts regular damage.
3,20,bind,Inflicts regular damage. For the next 2–5 tur...
4,22,vine-whip,Inflicts regular damage.
...,...,...,...
81,580,grassy-terrain,"For five turns, heals all Pokémon on the groun..."
82,590,confide,Lowers the target's Special Attack by one stage.
83,803,grassy-glide,Inflicts regular damage.
84,851,tera-blast,Null
